In [69]:
puzzle = open("AoC15.txt").read().splitlines()


In [70]:
grid = []
row = 0
while puzzle[row] != "":
    grid.append(list(puzzle[row]))
    row += 1
instructions = "".join(puzzle[row+1:])

In [71]:
def move(grid, curri, currj, dir):
    #move one step in the direction dir, accounting for obstacles and boxes
    #changes grid in place
    #returns new position
    dirs = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}
    i, j = dirs[dir]
    if grid[curri+i][currj+j] == "#":
        return curri, currj
    if grid[curri+i][currj+j] == ".":
        return curri+i, currj+j
    if grid[curri+i][currj+j] == "O":
        newPos = curri, currj
        while grid[newPos[0]+i][newPos[1]+j] == "O":
            newPos = newPos[0]+i, newPos[1]+j
        if grid[newPos[0]+i][newPos[1]+j] == "#":
            return curri, currj
        if grid[newPos[0]+i][newPos[1]+j] == ".":
            grid[newPos[0]+i][newPos[1]+j] = "O"
            grid[curri+i][currj+j] = "."
            return curri+i, currj+j
    return curri, currj

In [72]:
def iterate(grid, instructions, starti, startj):
    curri, currj = starti, startj
    for i in range(len(instructions)):
        curri, currj = move(grid, curri, currj, instructions[i])
    return grid
def GPS(grid):
    sum = 0
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == "O":
                sum += 100*i+j
    return sum

In [73]:
newGrid = [row.copy() for row in grid]
for i in range(len(grid)):
    for j in range(len(grid[i])):
        if grid[i][j] == "@":
            starti, startj = i, j
            newGrid[i][j] = "."
newGrid = iterate(newGrid, instructions, starti, startj)
ans1 = GPS(newGrid)
print(ans1)

1406392


In [74]:
def createNewGrid(grid):
    newGrid = []
    for i in range(len(grid)):
        newLine = []
        for j in range(len(grid[0])):
            if grid[i][j] == "@":
                newLine.append("@")
                newLine.append(".")
            elif grid[i][j] == "#":
                newLine.append("#")
                newLine.append("#")
            elif grid[i][j] == ".":
                newLine.append(".")
                newLine.append(".")
            elif grid[i][j] == "O":
                newLine.append("[")
                newLine.append("]")
        newGrid.append(newLine)
    return newGrid
def getBoxes(grid):
    boxes = set()
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == "[":
                boxes.add((i, j))
    return boxes
def getWalls(grid):
    walls = set()
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == "#":
                walls.add((i, j))
    return walls
def getGPS(boxes):
    sum = 0
    for i, j in boxes:
        sum += 100*i+j
    return sum
def getStart(grid):
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if grid[i][j] == "@":
                return i, j

In [75]:
def move2(boxes, walls, curri, currj, dir):
    #move one step in the direction dir, accounting for obstacles and boxes
    #returns new position, set of boxes
    dirs = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}
    i, j = dirs[dir]
    if (curri+i, currj+j) in walls:
        return curri, currj, boxes
    if (curri+i, currj+j) not in walls and (curri+i, currj+j) not in boxes and (curri+i, currj+j-1) not in boxes:
        return curri+i, currj+j, boxes
    if (curri+i, currj+j) in boxes or (curri+i, currj+j-1) in boxes:
        movingBoxes = set()
        if (curri+i, currj+j) in boxes:
            movingBoxes.add((curri+i, currj+j))
            newPos = curri+i, currj+j
        else:
            movingBoxes.add((curri+i, currj+j-1))
            newPos = curri+i, currj+j-1
        queue = [(newPos)]
        visited = set()
        ctr = 0
        while queue:
            ctr+=1
            box = queue.pop(0)
            if box in visited:
                continue
            visited.add(box)
            movingBoxes.add(box)
            if (box[0]+i, box[1]+j) in walls or (box[0]+i, box[1]+j+1) in walls:
                return curri, currj, boxes
            if (box[0]+i, box[1]+j) in boxes:
                if (box[0]+i, box[1]+j) not in visited:
                    queue.append((box[0]+i, box[1]+j))
            if (box[0]+i, box[1]+j+1) in boxes:
                if (box[0]+i, box[1]+j+1) not in visited:
                    queue.append((box[0]+i, box[1]+j+1))
            if (box[0]+i, box[1]+j-1) in boxes:
                if (box[0]+i, box[1]+j-1) not in visited:
                    queue.append((box[0]+i, box[1]+j-1))
        for m in movingBoxes:
            boxes.remove(m)
        for m in movingBoxes:
            boxes.add((m[0]+i, m[1]+j))
        return curri+i, currj+j, boxes


In [76]:
def createGrid(walls, boxes, maxI, maxJ, currI, currJ):
    grid = []
    for i in range(maxI):
        line = []
        j = 0
        while j < maxJ:
            if (i, j) in walls:
                line.append("#")
            elif (i, j) in boxes:
                line.append("[")
                line.append("]")
                j+=1
            elif (i, j) == (currI, currJ):
                line.append("@")
            else:
                line.append(".")
            j+=1
        grid.append(line)
    return grid


In [77]:
newGrid2 = createNewGrid(grid)
maxI, maxJ = len(newGrid2), len(newGrid2[0])
starti, startj = getStart(newGrid2)
boxes = getBoxes(newGrid2)
walls = getWalls(newGrid2)
curri, currj = starti, startj
from tqdm import tqdm
for i in tqdm(range(len(instructions))):
    curri, currj, boxes = move2(boxes, walls, curri, currj, instructions[i])
ans2 = getGPS(boxes)
print(ans2)

100%|██████████| 20000/20000 [00:00<00:00, 502929.22it/s]

1429013
